# Assignement 1 - Bio systems

### Task 1 - Visualize the reaction maximal activity data on the ESCHER map of the E. coli core model. For this we use the non-interactive version:https://escher.github.io/#/app?map=e_coli_core.Core%20metabolism&tool=Builder&scrollToZoom=true&model=e_coli_core. In order to load the data, go to “Data 🡪 “Load reaction data” and load the file. The data will then be visualized in the same way that we inspected actual flux predictions in the interactive session in Week 1.

-----------------------------------------------------------
#### a) In the interactive session, we saw reaction fluxes in a linear pathway being equal to each other due to mass balance constraints. Do you observe the same thing now for the maximal reaction activities? Explain your observations. [5 pt] 

<img src="ss_mm.png" width="50%">


answer: we are using the .csv file to attribute individually to every reaction its own maximum possible flux value (upper balance), which is different than the steady-state flux demo we were playing with in the interactive session, in which linear pathways showed equal flux values due to mass balance constraints

-----------------------------------------------------------------------------
#### b) Some reactions have no maximal reaction activity data. Identify at least two different kinds of such reactions, and explain why gene expression-derived data would not be applicable for these kinds of reactions. [5 pt]

answer:

two examples of reactions missing from the maximal reaction activity data:

**1.EX_glc__D_e (D-Glucose exchange/uptake)**

Why no gene expression data: This reaction is an exchange reaction, that represents a system boundary in the metabolic model, not a specific enzymatic reaction. It defines the interface between the extracellular compartment and the cellular one. While actual glucose transport involves specific transporters, the exchange reaction is purely a mathematical construct.

Gene association: Has zero associated genes because it's a model boundary condition, not a biochemical reaction catalyzed by a specific enzyme.

**2.ATPM (ATP maintenance requirement)**

Why no gene expression data: This reaction represents the overall cellular energy cost for maintaining basic cellular functions (protein turnover, DNA repair, etc) It's not catalyzed by a single enzyme, it just represents the overall ATP consumption from hundreds of different maintenance processes occurring simultaneously in the cell.
Gene association: Has zero associated genes because it's a mathematical representation of energy drain rather than a specific enzymatic reaction. You cannot measure gene expression for "cellular maintenance" as a whole - it's the sum of many different processes.


In conclusion: Gene expression-derived data can only be applied to reactions that are catalyzed by specific enzymes encoded by identifiable genes.

---------------------------------------------------------------------------
### Task 2 - We will now establish an enzyme activity-constrained metabolic model in COBRApy. Please implement the maximal reaction activity data in the E. coli core model from the practical per the below instructions.

In [1]:
import cobra
import pandas as pd
##!!!!dont forget to updae the path names according to yourselves!!!!
model = cobra.io.load_json_model('e_coli_core.json')
activity_df = pd.read_csv('e_coli_core_expression.csv')

#strip the csv a little
activity_df.columns = [c.strip() for c in activity_df.columns]
#kept the original bounds so that when we run the model again we dont use the modified bounds
#(in edge cases like ATPM or glucose exchange we wanna consider their original definition not a modified one)
orig_bounds = {reaction.id: (reaction.lower_bound, reaction.upper_bound) for reaction in model.reactions}

# QUESTION 2: Apply the instructions
for reaction in model.reactions:
    r_id = reaction.id

    ##consider edge cases first before applying the general rule because if not they’d get overridden like normal reactions which would not make sense biologically
    #Special case: glucose exchange
    if r_id == "EX_glc__D_e":
        reaction.lower_bound = -1000.0
        reaction.upper_bound = 1000.0
        continue

    #check if the reaction is in the CSV, if not return an empty. (if returned empty: no activity data for this reaction -≥ leave defaults unchanged)
    match = activity_df.loc[activity_df["# Reaction ID"] == r_id, "reaction activity [mmol/gDW/h]"]

    if not match.empty:
        value = abs(float(match.values[0]))  #force non-negative
        lb0, ub0 = orig_bounds[r_id]       #original bounds for (ir)reversability

        #Special case: ATPM
        if r_id == "ATPM":
            # Keep lower bound as-is, only cap upper bound
            reaction.upper_bound = value
            continue

        #Decide on the constraints based on original directionality
        if lb0 < 0 and ub0 > 0:         # reversible
            reaction.lower_bound = -value
            reaction.upper_bound = value
        elif lb0 >= 0 and ub0 > 0:      # forward-only irreversible
            reaction.lower_bound = 0.0
            reaction.upper_bound = value
        ##elif lb0 < 0 and ub0 <= 0:      # reverse-only irreversible (not sure if we are supposed to consider this as well for the irreversible direction)
            ##reaction.lower_bound = -value
            ##reaction.upper_bound = 0.0
        else: #this would be the degenerate case (lb==ub, blocked, etc.)
            pass

#gatyher the results in df
bounds_df = pd.DataFrame(
    [(reaction.id, float(reaction.lower_bound), float(reaction.upper_bound)) for reaction in model.reactions],
    columns=["# Reaction ID", "Lower Bound", "Upper Bound"]
).sort_values("# Reaction ID").reset_index(drop=True)

pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", "{:,.6g}".format)
print(bounds_df.to_string(index=False))

## DISCLAIMER: USE OF GENERATIVE AI
#used AI to strip the csv and preserve the original bounds of the model

           # Reaction ID  Lower Bound  Upper Bound
                   ACALD        -1.16         1.16
                  ACALDt        -2.29         2.29
                    ACKr        -1.19         1.19
                  ACONTa       -25.35        25.35
                  ACONTb       -25.35        25.35
                   ACt2r        -3.23         3.23
                    ADK1       -30.57        30.57
                   AKGDH            0        24.35
                  AKGt2r         -3.1          3.1
                  ALCD2x        -9.01         9.01
                    ATPM         8.39        1,000
                  ATPS4r        -60.5         60.5
BIOMASS_Ecoli_core_w_GAM            0        1,000
                    CO2t       -30.45        30.45
                      CS            0        20.56
                   CYTBD            0        40.56
                 D_LACt2        -4.56         4.56
                     ENO       -28.78        28.78
                 ETOHt2r       

---------------------------------------------------------------------------
### Task 3 - In addition to maximizing reaction rate (“flux”) through some reaction of biological interest, like the biomass reaction, we can use Flux Balance Analysis (FBA) optimizations also to ask: What are the smallest and largest fluxes possible per reaction, given all present constraints in the network? This is known as “Flux Variabiliry Analysis” or FVA. In FVA, we loop over every reaction of interest and once maximize and once minimize its flux via FBA; thus obtaining the permissible flux range for each reaction in the model.

a) Carry out an FVA for all reactions in the model and print out the resulting minimal and maximal fluxes per reaction (simple printout sufficient)

In [2]:
fva =cobra.flux_analysis.variability.flux_variability_analysis(model =model, reaction_list = model.reactions, loopless = False, fraction_of_optimum = 0.0)
pd.set_option("display.max_rows", None)
print(fva)

                              minimum      maximum
PFK                          0.334677      11.1536
PFL                                 0            1
PGI                        -0.0853226      9.02035
PGK                           -17.596    -0.879355
PGL                                 0      4.52239
ACALD                           -1.16            0
AKGt2r                           -3.1            0
PGM                          -17.0389    -0.879355
PIt2r                     4.78153e-15      1.78308
ALCD2x                          -1.16 -7.65727e-17
ACALDt                          -1.16            0
ACKr                            -1.19            0
PPC                                 0         2.56
ACONTa                              0      5.77556
ACONTb                              0      5.77556
ATPM                             8.39      39.9838
PPCK                                0         4.69
ACt2r                           -1.19            0
PPS                            

b) Identify any reactions with gene expression-imposed maximal reaction activities, whose permissible
flux range in forward direction is nonzero yet comes out less than its upper flux bound. State their number and explain why a set of reactions behaves this way. (You can ignore reaction FORt here which as seen
in the practical has a nonconventional direction definition.) [15 pt]

In [3]:
reac=[]
for reaction in model.reactions:
    min_flux = fva.loc[reaction.id, "minimum"]
    max_flux = fva.loc[reaction.id, "maximum"]
    if min_flux>0 and max_flux<reaction.upper_bound:
        reac.append((reaction.id, min_flux, max_flux, reaction.upper_bound ))
reac_df = pd.DataFrame(reac, columns=["reaction", "min flux", "max flux", "upper bound"])
print(reac_df)
print(f"In total {len(reac_df)} reactions")

    reaction    min flux  max flux  upper bound
0        PFK    0.334677   11.1536        12.12
1      PIt2r 4.78153e-15   1.78308         6.03
2       ATPM        8.39   39.9838        1,000
3     ATPS4r    0.426667    27.641         60.5
4      CYTBD           1    24.512        40.56
5        ENO    0.879355   17.0389        28.78
6        TPI    0.334677   8.80356        45.56
7     EX_h_e 1.20129e-14   21.8022        1,000
8   EX_h2o_e       0.538   18.5909        1,000
9        FBA    0.334677   8.80356        24.45
10      GAPD    0.879355    17.596        26.13
11    GLCpts    0.479429    9.4933        16.34
12    ICDHyr 7.97411e-16   5.20213        15.06
13       O2t         0.5    12.256        40.67
In total 14 reactions


Answer:
If the gene encoding the enzyme for a reaction is expressed at low levels, less enzyme is synthesized. As a result, the limited enzyme availability becomes a bottleneck. While the reaction can proceed at a non-zero rate, it will always be constrained below its maximum potential (upper flux bound) due to insufficient enzyme capacity to handle the flux.

c) How many reactions have a positive minimal flux in the FVA? State their number and explain why a
set of reactions behaves this way. [15 pt]


In [4]:
positive_min_flux = []
for reaction in model.reactions:
    min_flux = fva.loc[reaction.id, "minimum"]
    if min_flux>0:
        positive_min_flux.append(reaction.id)

print(positive_min_flux)
print(f"Number of reactions with positive minimal flux:{len(positive_min_flux)}")

['PFK', 'PIt2r', 'ATPM', 'ATPS4r', 'CYTBD', 'ENO', 'TPI', 'EX_h_e', 'EX_h2o_e', 'FBA', 'GAPD', 'GLCpts', 'ICDHyr', 'NADH16', 'NH4t', 'O2t']
Number of reactions with positive minimal flux:16


Answer:
A set of reactions that exhibit such behavior may be necessary to maintain critical cellular functions, such as metabolic balance, production of vital metabolites, or regulation of essential biochemical processes. As a result, these reactions are forced to maintain a positive flow even under minimal conditions so that the cell can continue to perform these important tasks.

---------------------------------------------------------------------------
### Task 4 - FBA optimization of biomass

a) Carry out an FBA optimization of biomass and report the maximal biomass production rate in the presence of the implemented constraints. 


In [16]:
solution = model.optimize() #default is biomass optimization

print(f"The maximal biomass production rate is: {solution.objective_value}")

The maximal biomass production rate is: 0.48470309797054556


b) Determine the “bottleneck(s)” – i. e. those reaction(s) whose flux reaches its maximum.

In [ ]:
bottlenecks = []

for reac in model.reactions:
    max_flux = fva.loc[reac.id, 'maximum']
    if solution.fluxes[reac.id] == max_flux and max_flux != 0: #remove reaction that are always negative
        bottlenecks.append((reac.id, solution.fluxes[reac.id], max_flux))
bottlenecks_df = pd.DataFrame(bottlenecks, columns=["reaction", "optimized flux", "maximum posisble flux"])
print(bottlenecks_df)

#not sure if this is needed but reactions with a negative flux can also be a bottle neck 
bottlenecks_min= []
for reac in model.reactions:
    min_flux = fva.loc[reac.id, 'minimum']
    if solution.fluxes[reac.id] == min_flux and min_flux != 0: #remove irreversible reactions
        bottlenecks_min.append((reac.id, solution.fluxes[reac.id], min_flux))
bottlenecks_min_df = pd.DataFrame(bottlenecks_min, columns=["reaction", "optimized flux", "minimum posisble flux"])
print(bottlenecks_min_df)

      reaction  optimized flux  maximum posisble flux
0          PFL               1                      1
1         PTAr            1.19                   1.19
2         THD2             4.5                    4.5
3      EX_ac_e            1.19                   1.19
4    EX_etoh_e            1.16                   1.16
5     EX_for_e               1                      1
6  EX_lac__D_e            2.07                   2.07
7     EX_pyr_e            1.26                   1.26
8       NADH16           20.26                  20.26
  reaction  optimized flux  minimum posisble flux
0    ACALD           -1.16                  -1.16
1   ALCD2x           -1.16                  -1.16
2     ACKr           -1.19                  -1.19
3     ATPM            8.39                   8.39
4    ACt2r           -1.19                  -1.19
5    PYRt2           -1.26                  -1.26
6  D_LACt2           -2.07                  -2.07
7  ETOHt2r           -1.16                  -1.16
8    LDH_D